# Searching for interactions in a social network

In this example, we will explore a social network (diaspora*: https://diasporafoundation.org/) and explore interactions between users. The analysis is done in a very naive way.

## Initialization

As a first step, we will import the necessary libraries.

In [ ]:
import matplotlib.pyplot as plt
import networkx as nx
import requests
from datetime import datetime

%matplotlib inline

Then we define some basic variables.

In [ ]:
url_main = 'https://diaspora.psyco.fr/tags/covid-19.json'
url_post_pattern = 'https://diaspora.psyco.fr/posts/{guid}.json'

## Collecting data

We are going to collect the interactions between users and record them in a graph. In this example, we do not take into account the type of interaction, nor the direction of information propagation.

In [ ]:
G = nx.Graph()
max_time = None

for _ in range(10):
    if max_time is None:
        url_flux = url_main
    else:
        url_flux = f'{url_main}?max_time={max_time}'
    print(f'Parsing {url_flux}')
    r_main = requests.get(url_flux)
    if r_main.status_code  != 200:
        print(f'Failed to retrieve main flux: {r_main.status_code}')
        break
    messages = r_main.json()
    for message in messages:
        interactions_count = message['interactions']['comments_count'] + message['interactions']['likes_count'] + message['interactions']['reshares_count']
        if interactions_count > 0:
            url_message = url_post_pattern.format(guid=message['guid'])
            print(f'Interractions found. Parsing {url_message}')
            r_message = requests.get(url_message)
            if r_message.status_code != 200:
                print(f'Failed to retrieve message: {r_message.status_code}')
                continue
            message_data = r_message.json()
            message_author = message_data['author']['diaspora_id']
            for interaction in ['likes', 'reshares', 'comments']:
                for item in message_data['interactions'][interaction]:
                    item_author = item['author']['diaspora_id']
                    G.add_edge(message_author, item_author)
    max_time = int(datetime.strptime(messages[-1]['created_at'], '%Y-%m-%dT%H:%M:%S.000Z').timestamp())
    print(f'next max_time: {max_time}')

## Analysis

Once the data has been collected, we can analyze it to find people of interest.

First, we visualize the graph in order to highlight the communities.

In [ ]:
fig = plt.figure(figsize=(24,12))
nx.draw(G)
plt.show()

The visualization of the graph tells us that there is one main community and different anecdotal communities. We will focus on the main community and look for people who have the most interactions.

In [ ]:
component = sorted(nx.connected_components(G), key=lambda x : len(x), reverse=True)[0]
subG = nx.subgraph(G, component)
plt.figure(figsize=(24,12))
nx.draw(subG, with_labels=True)
plt.show()
centrality = nx.degree_centrality(subG)
users = sorted(centrality.keys(), key=lambda x: centrality[x], reverse=True)[:5]
for user in users:
    username, pod = user.split('@')
    user_url = f'https://{pod}/u/{username}'
    print(user, centrality[user], user_url)

--------------------------

&copy; Sébastien Adam 2020

* Website: https://www.sebastienadam.be
* diaspora*: https://diasp.de/u/sebastienadam
* LinkedIn: https://www.linkedin.com/in/sebastien-adam-be/